In [46]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from einops.layers.torch import Rearrange
from einops import rearrange
from facenet_pytorch import MTCNN
from self_attention_cv import TransformerEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import face_alignment
import requests
import warnings
warnings.filterwarnings("ignore")

In [47]:
!ls 'Videos7439' | wc -l

7439


In [48]:
!ls 'Spectrograms7439' | wc -l

7439


In [49]:
videos_folder = 'Videos7439'
spectrograms_folder = 'Spectrograms7439'

In [50]:
from torchvision import models

class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = models.vgg16(pretrained=True)
        # Modify the first layer to accept 1 channel input (for grayscale spectrograms)
        self.features.features[0] = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Modify the final layer to output desired feature size
        self.features.classifier[6] = nn.Linear(self.features.classifier[6].in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.features(x)
        x = self.softmax(x)
        return x

In [51]:
def extract_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    mid_frame_index = frame_count // 2  # Index of the frame in the middle of the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_index)
    ret, frame = cap.read()
    if ret:
        cap.release()
        return frame
    else:
        cap.release()
        return None

In [52]:
def detect_face(frame):
    mtcnn = MTCNN()
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        # Assuming only one face in the frame
        box = boxes[0]
        x1, y1, x2, y2 = box
        # Crop the frame to the detected face
        cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
        return cropped_frame
    else:
        return None

In [53]:
def preprocess_image(frame):
    # Convert the frame to a PIL Image
    frame_pil = Image.fromarray(frame.astype('uint8'))

    # Convert the image to RGB by duplicating channels
    frame_pil = frame_pil.convert('RGB')

    # Resize and normalize the frame
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization for RGB
    ])
    img_tensor = transform(frame_pil)
    return img_tensor

In [54]:
def preprocess_spectrogram(image_path):
    img = Image.open(image_path).convert('RGB')  # Convert to RGB by duplicating channels
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match VGG input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard normalization for RGB
    ])
    img_tensor = transform(img)
    return img_tensor

In [55]:
def load_dataset(videos_folder):
    X = []
    y = []
    skipped_files = []
    video_files = [file for file in sorted(os.listdir(videos_folder)) if file.endswith(".flv")]
    for video_file in tqdm(video_files):
        video_path = os.path.join(videos_folder, video_file)
        frame = extract_frame(video_path)
        if frame is not None:
            cropped_face = detect_face(frame)
            if cropped_face is not None:
                preprocessed_face = preprocess_image(cropped_face)
                X.append(preprocessed_face)
                label = video_file.split("_")[2].split(".")[0]  # Adjusted to handle different file extensions
                if label == "HAP":
                    y.append(0)
                elif label == "SAD":
                    y.append(1)
                elif label == "ANG":
                    y.append(2)
                elif label == "DIS":
                    y.append(3)
                elif label == "FEA":
                    y.append(4)
                elif label == "NEU":
                    y.append(5)
            else:
                print(f"No face detected in {video_file}. Skipping.")
                skipped_files.append(video_file[:-3])
        else:
            print(f"Failed to extract frame from {video_file}. Skipping.")
            skipped_files.append(video_file[:-3])
    return X, y, skipped_files

In [56]:
def load_spectrogram_dataset(spectrograms_folder, skipped_files):
    X = []
    y = []
    # List all files in the input folder
    files = sorted(os.listdir(spectrograms_folder))
    # Iterate over files in the folder
    for filename in tqdm(files):
        if filename.endswith(".png") and filename[:-3] not in skipped_files:  # Assuming mel spectrograms are stored as PNG files
            input_path = os.path.join(spectrograms_folder, filename)
            img_tensor = preprocess_spectrogram(input_path)
            X.append(img_tensor)
            # Extract label from filename (assuming filename is in format "abc_IEO_label_xyz.png")
            label = filename.split("_")[2]
            if label == "HAP":
                y.append(0)
            elif label == "SAD":
                y.append(1)
            elif label == "ANG":
                y.append(2)
            elif label == "DIS":
                y.append(3)
            elif label == "FEA":
                y.append(4)
            elif label == "NEU":
                y.append(5)
    return X, y

In [57]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [58]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [59]:
assert([file[:-3] for file in sorted(os.listdir(videos_folder))] == [file[:-3] for file in sorted(os.listdir(spectrograms_folder))])

In [60]:
X, y, skipped_files = load_dataset(videos_folder)
#X_spec, y_spec = load_spectrogram_dataset(spectrograms_folder, skipped_files)

 35%|███▌      | 2640/7439 [02:50<04:57, 16.16it/s]

No face detected in 1033_ITH_HAP_XX.flv. Skipping.


 36%|███▌      | 2664/7439 [02:52<04:48, 16.53it/s]

No face detected in 1033_MTI_HAP_XX.flv. Skipping.


 36%|███▌      | 2680/7439 [02:53<04:47, 16.58it/s]

...

 87%|████████▋ | 6480/7439 [06:53<01:01, 15.53it/s]

No face detected in 1080_IOM_ANG_XX.flv. Skipping.


 88%|████████▊ | 6532/7439 [06:56<01:00, 15.11it/s]

No face detected in 1080_TSI_HAP_XX.flv. Skipping.


100%|██████████| 7439/7439 [07:54<00:00, 15.66it/s]


In [61]:
np.save('X.npy', np.array(X))
np.save('y.npy', np.array(y))


In [62]:
#X_spec_rgb = np.repeat(X_spec, 3, axis=1)  # Repeat the channels dimension
print(f"Total number of samples: {len(X)}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Number of train samples: {len(X_train)}", f"Number of test samples: {len(X_test)}")

Total number of samples: 7429
Number of train samples: 5200 Number of test samples: 2229


In [63]:
if __name__ == "__main__":
    # Check if input arguments are provided
    # if len(sys.argv) != 2:
    #     print("Usage: python melspec_to_features_cnn.py input_folder")
    #     sys.exit(1)

    # input_folder = sys.argv[1]
    # spectrograms_folder = 'melspec_fullscale'
    # Check if input folder exists

    # Load dataset and split into train and test sets
    # Initialize the model
    model = CNN(num_classes=6)  # 3 classes for HAPPY, SAD, ANGRY
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define loss function and optimizer
    _lr = 0.00001
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=_lr)

    # Create data loaders
    _bs = 16
    train_loader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=_bs, shuffle=True)
    test_loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=_bs)
    print(f"Batch size: {_bs}", f"lr: {_lr}")
    # Training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        print("Epoch " + str(epoch))
        train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
        test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Batch size: 16 lr: 1e-05
Epoch 0


100%|██████████| 140/140 [00:02<00:00, 59.41it/s]


Epoch 1/50, Train Loss: 1.7567, Train Accuracy: 0.2338, Test Loss: 1.6501, Test Accuracy: 0.3616
Epoch 1


100%|██████████| 140/140 [00:02<00:00, 60.08it/s]


Epoch 2/50, Train Loss: 1.6272, Train Accuracy: 0.3971, Test Loss: 1.5871, Test Accuracy: 0.4424
Epoch 2


...

Epoch 48/50, Train Loss: 1.0996, Train Accuracy: 0.9450, Test Loss: 1.3555, Test Accuracy: 0.6815
Epoch 48


100%|██████████| 140/140 [00:02<00:00, 59.40it/s]


Epoch 49/50, Train Loss: 1.1031, Train Accuracy: 0.9415, Test Loss: 1.3591, Test Accuracy: 0.6783
Epoch 49


100%|██████████| 140/140 [00:02<00:00, 58.87it/s]

Epoch 50/50, Train Loss: 1.1049, Train Accuracy: 0.9404, Test Loss: 1.3641, Test Accuracy: 0.6765


In [64]:
torch.save(model.state_dict(), 'vgg16_video_'+str(num_epochs)+'_'+str(_bs)+'_'+str(_lr))

In [ ]:
# ! ls -lh /content/